In [20]:
import os
import json
import random
import pandas as pd
from pathlib import Path
import json

folder_aws_path = "aws/"
folder_build_path = "build/"
folder_build_config_path = "build/config/"
folder_build_task_path = "build/task/"
folder_build_mturk_path = "build/mturk/"
folder_build_deploy_path = "build/deploy/"
folder_build_skeleton_path = "build/skeleton/"
folder_tasks_path = "tasks/"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4, default=str)
        f.close()

def remove_json(folder, filename):
    os.remove(f"{folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        return data
    else:
        return {}

## Section 1 - Environment variables loading

Remember to restart Jupyter if you edit environment variables values

In [21]:
from dotenv import load_dotenv
from distutils.util import strtobool

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

task_name = os.getenv('task_name')
batch_name = os.getenv('batch_name')
admin_user = os.getenv('admin_user')
admin_password = os.getenv('admin_password')
deploy_config = strtobool(os.getenv('deploy_config').lower())

aws_account_id = os.getenv('aws_account_id')
aws_sdk_access_id = os.getenv('aws_sdk_access_id')
aws_sdk_access_secret = os.getenv('aws_sdk_access_secret')

aws_region = os.getenv('aws_region')
aws_private_bucket = os.getenv('aws_private_bucket')
aws_deploy_bucket = os.getenv('aws_deploy_bucket')

bing_api_key = os.getenv('bing_api_key')

# print("Printing env vars values:")
# print(f"AWS ACCOUNT ID: {aws_account_id}")
# print(f"AWS SDK ACCESS ID: {aws_sdk_access_id}")
# print(f"AWS SDK ACCESS SECRET: {aws_sdk_access_secret}")
# print(f"AWS REGION: {aws_region}")
# print(f"AWS PRIVATE BUCKET: {aws_private_bucket}")
# print(f"AWS DEPLOY BUCKET: {aws_deploy_bucket}")
# print(f"BING API KEY: {bing_api_key}")
# print(f"TASK NAME: {task_name}")
# print(f"BATCH NAME: {batch_name}")

## Section 2 - Setting up IAM policies and identity
### build/aws

In [4]:
import boto3
from botocore.exceptions import ClientError

iam = boto3.client('iam')

crowd_workers_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "allowWorkerInteraction",
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{aws_private_bucket}",
                f"arn:aws:s3:::{aws_private_bucket}/*",
                f"arn:aws:s3:::{aws_deploy_bucket}",
                f"arn:aws:s3:::{aws_deploy_bucket}/*"
            ]
        }
    ]
}

policy = None
try:
    policy = iam.create_policy(
        PolicyName='crowd-workers-dev',
        PolicyDocument=json.dumps(crowd_workers_policy)
    )
    print(
        f"Crowd workers IAM policy with ARN {policy['Policy']['Arn']} creation completed, HTTP STATUS CODE: {policy['ResponseMetadata']['HTTPStatusCode']}.")
except (iam.exceptions.NoSuchEntityException, iam.exceptions.EntityAlreadyExistsException) as exception:
    policy = iam.get_policy(PolicyArn=f"arn:aws:iam::{aws_account_id}:policy/crowd-workers-dev")
    print(
        f"Crowd workers IAM policy with ARN {policy['Policy']['Arn']} retrieved, HTTP STATUS CODE: {policy['ResponseMetadata']['HTTPStatusCode']}.")
serialize_json(folder_aws_path, f"policy_{policy['Policy']['PolicyName']}.json", policy)

user = None
try:
    user = iam.create_user(UserName="worker-dev")
    print(
        f"Crowd workers user {user['User']['UserName']} created, HTTP STATUS CODE: {user['ResponseMetadata']['HTTPStatusCode']}.")
except (iam.exceptions.NoSuchEntityException, iam.exceptions.EntityAlreadyExistsException) as exception:
    user = iam.get_user(UserName="worker-dev")
    print(
        f"Crowd worker user {user['User']['UserName']} retrieved, HTTP STATUS CODE: {user['ResponseMetadata']['HTTPStatusCode']}.")
serialize_json(folder_aws_path, f"user_{user['User']['UserName']}_data.json", user)

response = iam.attach_user_policy(UserName=user['User']['UserName'], PolicyArn=policy['Policy']['Arn'])
policy = iam.get_policy(PolicyArn=f"{policy['Policy']['Arn']}")
print(
    f"Policy with ARN {policy['Policy']['Arn']} attached to user {user['User']['UserName']}, HTTP STATUS CODE: {user['ResponseMetadata']['HTTPStatusCode']}")

keys = []
paginator = iam.get_paginator('list_access_keys')
for found_keys in paginator.paginate(UserName=user['User']['UserName']):
    for (index, key) in enumerate(found_keys['AccessKeyMetadata']):
        keyData = read_json(f"{folder_aws_path}user_{user['User']['UserName']}_access_key_{key['AccessKeyId']}.json")
        if keyData:
            keys.append(keyData)
        else:
            response = iam.delete_access_key(UserName=user['User']['UserName'], AccessKeyId=key['AccessKeyId'])
            print(f"Key {index} data not found on disk; deleting it on AWS, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}")

if len(keys) < 2:
    key = iam.create_access_key(UserName=user['User']['UserName'])
    serialize_json(folder_aws_path, f"user_{user['User']['UserName']}_access_key_{key['AccessKey']['AccessKeyId']}.json", key)
    print(f"Access key with for user {user['User']['UserName']} created, HTTP STATUS CODE: {key['ResponseMetadata']['HTTPStatusCode']}.")
    keys.append(key)
    if not os.path.exists(f"{folder_aws_path}user_{user['User']['UserName']}_access_key_{key['AccessKey']['AccessKeyId']}.json"):
        serialize_json(folder_aws_path, f"user_{user['User']['UserName']}_access_key_{key['AccessKey']['AccessKeyId']}.json", key)
        print(f"Access key for user {user['User']['UserName']} created, HTTP STATUS CODE: {key['ResponseMetadata']['HTTPStatusCode']}.")

key_selected = random.choice(keys)
key_data = read_json(f"{folder_aws_path}user_{user['User']['UserName']}_access_key_{key_selected['AccessKey']['AccessKeyId']}.json")

print("Key data found on disk and loaded.")

aws_worker_access_id = key_data['AccessKey']['AccessKeyId']
aws_worker_access_secret = key_data['AccessKey']['SecretAccessKey']


Crowd workers IAM policy with ARN arn:aws:iam::269559900417:policy/crowd-workers-dev retrieved, HTTP STATUS CODE: 200.
Crowd worker user worker-dev retrieved, HTTP STATUS CODE: 200.
Policy with ARN arn:aws:iam::269559900417:policy/crowd-workers-dev attached to user worker-dev, HTTP STATUS CODE: 200
Access key with for user worker-dev created, HTTP STATUS CODE: 200.
Key data found on disk and loaded.


## Section 3 - Private and deploy bucket creation
### build/aws

In [5]:
s3_client = boto3.client('s3', aws_access_key_id=aws_sdk_access_id, aws_secret_access_key=aws_sdk_access_secret)
s3_resource = boto3.resource('s3')

buckets = []
for bucket in s3_resource.buckets.all():
    buckets.append(bucket.name)

print(f"---------- INITIALIZING PRIVATE BUCKET {aws_private_bucket} ----------")

try:
    private_bucket = s3_client.create_bucket(
        Bucket=aws_private_bucket,
        CreateBucketConfiguration={
            'LocationConstraint': aws_region
        }
    )
    print(f"Bucket creation completed, HTTP STATUS CODE: {private_bucket['ResponseMetadata']['HTTPStatusCode']}.")
except s3_client.exceptions.BucketAlreadyOwnedByYou as error:
    private_bucket = s3_resource.Bucket(aws_private_bucket)
    print(f"Bucket already present, HTTP STATUS CODE: {error.response['ResponseMetadata']['HTTPStatusCode']}.")
serialize_json(folder_aws_path, f"bucket_{aws_private_bucket}.json", private_bucket)

response = s3_client.put_public_access_block(
    Bucket=aws_private_bucket,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': True,
        'IgnorePublicAcls': True,
        'BlockPublicPolicy': True,
        'RestrictPublicBuckets': True
    },
)
print(f"Public access blocked, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")

private_bucket_policy = {
    "Version": "2012-10-17",
    "Id": "private-bucket-policy",
    "Statement": [
        {
            "Sid": "allow-bucket-interaction",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{aws_account_id}:user/{user['User']['UserName']}"
            },
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{aws_private_bucket}",
                f"arn:aws:s3:::{aws_private_bucket}/*"
            ]
        }
    ]
}

try:
    policy = s3_client.get_bucket_policy(Bucket=aws_private_bucket)
    policy['Policy'] = json.loads(policy['Policy'])
    print(f"Policy already present, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchBucketPolicy':
        response = s3_client.put_bucket_policy(Bucket=aws_private_bucket, Policy=json.dumps(private_bucket_policy))
        print(f"Policy configuration completed, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
    policy = s3_client.get_bucket_policy(Bucket=aws_private_bucket)
    policy['Policy'] = json.loads(policy['Policy'])
serialize_json(folder_aws_path, f"bucket_{aws_private_bucket}_policy.json", policy)

cors_configuration = {
    'CORSRules': [{
        'AllowedHeaders': ['*'],
        'AllowedMethods': ['GET', 'HEAD', 'PUT'],
        'AllowedOrigins': ['*'],
        'ExposeHeaders': [],
        'MaxAgeSeconds': 3000
    }]
}

try:
    cors_configuration = s3_client.get_bucket_cors(Bucket=aws_private_bucket)
    print(f"CORS Configuration already present, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchCORSConfiguration':
        response = s3_client.put_bucket_cors(Bucket=aws_private_bucket, CORSConfiguration=cors_configuration)
        print(f"CORS configuration completed, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
    cors_configuration = s3_client.get_bucket_cors(Bucket=aws_private_bucket)
    cors_configuration['CORSRules'] = json.loads(cors_configuration['CORSRules'])
    print(cors_configuration)
serialize_json(folder_aws_path, f"bucket_{aws_private_bucket}_cors.json", cors_configuration)

print(f"---------- INITIALIZATION COMPLETED ----------")

print(f"---------- INITIALIZING DEPLOY BUCKET {aws_deploy_bucket} ----------")

try:
    deploy_bucket = s3_client.create_bucket(
        Bucket=aws_deploy_bucket,
        CreateBucketConfiguration={
            'LocationConstraint': aws_region
        }
    )
    print(f"Bucket creation completed, HTTP STATUS CODE: {deploy_bucket['ResponseMetadata']['HTTPStatusCode']}.")
except s3_client.exceptions.BucketAlreadyOwnedByYou as error:
    deploy_bucket = s3_resource.Bucket(aws_deploy_bucket)
    print(f"Bucket already present, HTTP STATUS CODE: {error.response['ResponseMetadata']['HTTPStatusCode']}.")
serialize_json(folder_aws_path, f"bucket_{aws_deploy_bucket}.json", deploy_bucket)

deploy_bucket_policy = {
    "Version": "2012-10-17",
    "Id": "deploy-bucket-policy",
    "Statement": [
        {
            "Sid": "allow-bucket-interaction",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{aws_account_id}:user/{user['User']['UserName']}"
            },
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{aws_deploy_bucket}",
                f"arn:aws:s3:::{aws_deploy_bucket}/*"
            ]
        }
    ]
}

try:
    policy = s3_client.get_bucket_policy(Bucket=aws_deploy_bucket)
    policy['Policy'] = json.loads(policy['Policy'])
    print(f"Policy already present, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchBucketPolicy':
        response = s3_client.put_bucket_policy(Bucket=aws_deploy_bucket, Policy=json.dumps(deploy_bucket_policy))
        print(f"Policy configuration completed, HTTP STATUS CODE: {response['ResponseMetadata']['HTTPStatusCode']}.")
    policy = s3_client.get_bucket_policy(Bucket=aws_deploy_bucket)
    policy['Policy'] = json.loads(policy['Policy'])
serialize_json(folder_aws_path, f"bucket_{aws_private_bucket}_policy.json", policy)

print(f"---------- INITIALIZATION COMPLETED ----------")

---------- INITIALIZING PRIVATE BUCKET private-bucket-test-fdssfsdfdssdds ----------
Bucket already present, HTTP STATUS CODE: 409.
Public access blocked, HTTP STATUS CODE: 200.
Policy already present, HTTP STATUS CODE: 200.
CORS Configuration already present, HTTP STATUS CODE: 200.
---------- INITIALIZATION COMPLETED ----------
---------- INITIALIZING DEPLOY BUCKET deploy-bucket-test-fsfddfsfdsfsfsffssfdfsdfsfsdfsd ----------
Bucket already present, HTTP STATUS CODE: 409.
Policy already present, HTTP STATUS CODE: 200.
---------- INITIALIZATION COMPLETED ----------


# Section 4 - Environment file generation
### build/environments

In [6]:
environment_development = f"{folder_build_path}environments/environment.ts"
environment_production = f"{folder_build_path}environments/environment.prod.ts"

environment_dict = {
    "production": 'true',
    "configuration_local": 'false',
    "taskName": task_name,
    "batchName": batch_name,
    "region": aws_region,
    "bucket": aws_private_bucket,
    "aws_id_key": aws_worker_access_id,
    "aws_secret_key": aws_worker_access_secret,
    "bing_api_key": bing_api_key
}

with open(environment_production, 'w') as file:
    print("export const environment = {", file=file)
    for (env_var, value) in environment_dict.items():
        if env_var == 'production' or env_var == 'configuration_local':
            print(f"\t{env_var}: {value},", file=file)
        else:
            print(f"\t{env_var}: \"{value}\",", file=file)
    print("};", file=file)

print("File for PRODUCTION environment generated at path:", environment_production)

environment_dict = {
    "production": 'false',
    "configuration_local": 'true',
    "taskName": task_name,
    "batchName": batch_name,
    "region": aws_region,
    "bucket": aws_private_bucket,
    "aws_id_key": aws_worker_access_id,
    "aws_secret_key": aws_worker_access_secret,
    "bing_api_key": bing_api_key
}

with open(environment_development, 'w') as file:
    print("export const environment = {", file=file)
    for (env_var, value) in environment_dict.items():
        if env_var == 'production' or env_var == 'configuration_local':
            print(f"\t{env_var}: {value},", file=file)
        else:
            print(f"\t{env_var}: \"{value}\",", file=file)
    print("};", file=file)

print("File for DEVELOPMENT environment generated at path:", environment_development)

File for PRODUCTION environment generated at path: build/environments/environment.prod.ts
File for DEVELOPMENT environment generated at path: build/environments/environment.ts


## Section 5 - admin.json file generation
### Reference: https://pycryptodome.readthedocs.io/en/latest/src/cipher/classic.html#cbc-mode
### build/config

In [7]:
from base64 import b64encode
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

if not os.path.exists(folder_build_config_path):
    os.makedirs(folder_build_config_path, exist_ok=True)

admin_file = f"{folder_build_config_path}admin.json"

data = {"username": admin_user}
key = admin_password.encode()
cipher = AES.new(key, AES.MODE_CBC)
ct_bytes = cipher.encrypt(pad(json.dumps(data).encode(), AES.block_size))
init_vector = b64encode(cipher.iv).decode('utf-8')
cypher_text = b64encode(ct_bytes).decode('utf-8')
data = [{"crypt": cypher_text}]
with open(admin_file, 'w') as f:
    json.dump(data, f, indent=4)

print("admin.json file generated at path:", admin_file)

admin.json file generated at path: build/config/admin.json


## Section 6 - document.ts file generation
### build/skeleton

In [8]:
import textwrap

hits_file = f"{folder_build_task_path}hits.json"
document_interface = f"{folder_build_skeleton_path}document.ts"
if not os.path.exists(folder_build_skeleton_path):
    os.makedirs(folder_build_skeleton_path, exist_ok=True)

print(f"Reading hits file from path:")
print(hits_file)
hits = read_json(hits_file)
sample_element = hits.pop()['documents'].pop()

if not 'id' in sample_element.keys():
    raise Exception("Please, make sure that your hits.json file contains an attributed called \"id\"")

# This class provides a representation of a single document stored in single hit stored in the Amazon S3 bucket.
# The attribute <document_index> is additional and should not be touched and passed in the constructor.
# Each field of such Document must be mapped to an attribute of this class and set up in the constructor as it is shown.

with open(document_interface, 'w') as file:
    print("export class Document {", file=file)
    print("", file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t', subsequent_indent='\t\t')
    print(wrapper.fill("index: number;"), file=file)
    print(wrapper.fill("countdownExpired: boolean;"), file=file)
    for attribute, value in sample_element.items():
        try:
            element = json.loads(value)
            if isinstance(element, dict):
                print(wrapper.fill(f"{attribute}: Array<JSON>;"), file=file)
            elif isinstance(element, int) or isinstance(element, float):
                print(wrapper.fill(f"{attribute}: number;"), file=file)
            elif isinstance(element, list):
                print(wrapper.fill(f"{attribute}: Array<String>;"), file=file)
            else:
                print(wrapper.fill(f"{attribute}: string;"), file=file)
            print(f"Attribute with name: {attribute} and type: {type(element)} found")
        except (TypeError, ValueError) as e:
            if isinstance(value, list):
                print(wrapper.fill(f"{attribute}: Array<String>;"), file=file)
            elif isinstance(value, int) or isinstance(value, float):
                print(wrapper.fill(f"{attribute}: number;"), file=file)
            else:
                print(wrapper.fill(f"{attribute}: string;"), file=file)
            print(f"Attribute with name: {attribute} and type: {type(value)} found")
    print("", file=file)
    print(wrapper.fill(f"constructor ("), file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t\t', subsequent_indent='\t\t\t')
    print(wrapper.fill("index: number,"), file=file)
    print(wrapper.fill("data: JSON"), file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t', subsequent_indent='\t\t')
    print(wrapper.fill(") {"), file=file)
    print("", file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t\t', subsequent_indent='\t\t\t')
    print(wrapper.fill("this.index = index"), file=file)
    for attribute, value in sample_element.items():
        try:
            element = json.loads(value)
            if isinstance(element, dict):
                print(wrapper.fill(f"this.{attribute} = new Array<JSON>()"), file=file)
                print(wrapper.fill(f"for (let index = 0; index < data[\"{attribute}\"].length; index++) this.{attribute}.push(data[\"{attribute}\"][index])"), file=file)
            elif isinstance(element, list):
                print(wrapper.fill(f"this.{attribute} = new Array<String>()"), file=file)
                print(wrapper.fill(f"for (let index = 0; index < data[\"{attribute}\"].length; index++) this.{attribute}.push(data[\"{attribute}\"])"), file=file)
            else:
                wrapper = textwrap.TextWrapper(initial_indent='\t\t\t', subsequent_indent='\t\t\t')
                print(wrapper.fill(f"this.{attribute} = data[\"{attribute}\"]"), file=file)
        except (TypeError, ValueError) as e:
            if isinstance(value, list):
                print(wrapper.fill(f"this.{attribute} = new Array<String>()"), file=file)
                print(wrapper.fill(f"for (let index = 0; index < data[\"{attribute}\"].length; index++) this.{attribute}.push(data[\"{attribute}\"])"), file=file)
            else:
                wrapper = textwrap.TextWrapper(initial_indent='\t\t\t', subsequent_indent='\t\t\t')
                print(wrapper.fill(f"this.{attribute} = data[\"{attribute}\"]"), file=file)
    wrapper = textwrap.TextWrapper(initial_indent='\t\t', subsequent_indent='\t\t')
    print("", file=file)
    print(wrapper.fill("}"), file=file)
    print("", file=file)
    print("}", file=file)

print("Document interface built at path: ")
print(document_interface)

Reading hits file from path:
build/task/hits.json
Attribute with name: filename and type: <class 'str'> found
Attribute with name: title and type: <class 'str'> found
Attribute with name: subject and type: <class 'str'> found
Attribute with name: type and type: <class 'str'> found
Attribute with name: number and type: <class 'int'> found
Attribute with name: year and type: <class 'int'> found
Attribute with name: editorial_code and type: <class 'str'> found
Attribute with name: gazette_reference and type: <class 'str'> found
Attribute with name: publication_date and type: <class 'str'> found
Attribute with name: gazette_date and type: <class 'str'> found
Attribute with name: valid_from and type: <class 'str'> found
Attribute with name: last_updated and type: <class 'str'> found
Attribute with name: link_gazette and type: <class 'str'> found
Attribute with name: link_urn_nir and type: <class 'str'> found
Attribute with name: link_eli_id and type: <class 'str'> found
Attribute with name:

## Section 7 - Amazon Mechanical Turk assets generation
### build/mturk

In [9]:
from mako.template import Template

model = Template(filename=f"{folder_build_mturk_path}model.html")
mturk_page = model.render(
    aws_region = aws_region,
    aws_deploy_bucket = aws_deploy_bucket,
    task_name = task_name,
    batch_name = batch_name
)
mturk_page_file = f"{folder_build_mturk_path}index.html"
with open(mturk_page_file, 'w') as file:
    print(mturk_page, file=file)

print(f"Amazon Mechanical Turk landing page generated at path:")
print(mturk_page_file)

hits_file = f"{folder_build_task_path}hits.json"
mturk_tokens_file = f"{folder_build_mturk_path}tokens.csv"
print(f"Reading hits file from path:")
print(hits_file)
hits = read_json(hits_file)
token_df = pd.DataFrame(columns=["token_input", "token_output"])
for hit in hits:
    token_df = token_df.append({
        "token_input": hit['token_input'],
        "token_output": hit['token_output']
    }, ignore_index=True)
token_df.to_csv(mturk_tokens_file, index=False)
print(f"Tokens for {len(hits)} hits required by Amazon Mechanical Turk generated at path:")
print(mturk_tokens_file)

Amazon Mechanical Turk landing page generated at path:
build/mturk/index.html
Reading hits file from path:
build/task/hits.json
Tokens for 125 hits required by Amazon Mechanical Turk generated at path:
build/mturk/tokens.csv


## Section 8 - Angular Application Build & Packaging
### build/deploy

In [27]:
import subprocess
import shutil
from mako.template import Template

folder_build_result = f"../dist/"

print(f"---------- BUILD FOR TASK {task_name}/{batch_name} STARTED ----------")

print("Building Angular application with command:")
command = "ng build --configuration=\"production\" --output-hashing=none"
print(command)
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
for line in process.stdout:
    line_clean = line.decode().strip()
    if "Initial Total" in line_clean:
        line_clean = line_clean[2:]
    if line_clean!="":
        print(line_clean)
process.wait()

print("Merging Javascript assets to deploy folder:")
script_merged_file = f"{folder_build_deploy_path}scripts.js"
es_scripts = [
    'main.js',
    'polyfills.js',
    'runtime.js'
]
with open(script_merged_file, 'w') as outfile:
    for file in es_scripts:
        script_current_file = f"{folder_build_result}Crowd_Frame/{file}"
        print(f"Processing file: {script_current_file}")
        with open(script_current_file) as script:
            for line in script:
                outfile.write(line)
print(f"Path: {script_merged_file}")

print("Merging CSS assets to deploy folder:")
styles_merged_file = f"{folder_build_deploy_path}styles.css"
css_styles = ['styles.css',]
with open(styles_merged_file, 'w') as outfile:
    for file in css_styles:
        style_current_file = f"{folder_build_result}Crowd_Frame/{file}"
        print(f"Processing file: {style_current_file}")
        with open(style_current_file) as style:
            for line in style:
                outfile.write(line)
print(f"Path: {styles_merged_file}")

print("Deleting build result folder:")
print(f"Path: {folder_build_result}")
shutil.rmtree(folder_build_result)

model = Template(filename=f"{folder_build_deploy_path}model.html")
index_page = model.render(
    task_name = task_name,
    batch_name = batch_name
)
index_page_file = f"{folder_build_deploy_path}index.html"
with open(index_page_file, 'w') as file:
    print(index_page, file=file)

print("index.html page generated:")
print(f"Path: {index_page_file}")

print(f"---------- BUILD FOR TASK {task_name}/{batch_name} COMPLETED ----------")

---------- BUILD FOR TASK ItalianLaws/Test-1 STARTED ----------
Building Angular application with command:
ng build --configuration="production" --output-hashing=none
Initial Chunk Files | Names         |      Size
main.js             | main          |   4.95 MB
styles.css          | styles        | 144.13 kB
polyfills.js        | polyfills     |  44.17 kB
runtime.js          | runtime       |   1.10 kB
Initial Total |   5.14 MB
Build at: 2021-05-28T14:26:47.215Z - Hash: 435ca07744231163efd1 - Time: 42098ms
Merging Javascript assets to deploy folder:
Processing file: ../dist/Crowd_Frame/main.js
Processing file: ../dist/Crowd_Frame/polyfills.js
Processing file: ../dist/Crowd_Frame/runtime.js
Path: build/deploy/scripts.js
Merging CSS assets to deploy folder:
Processing file: ../dist/Crowd_Frame/styles.css
Path: build/deploy/styles.css
Deleting build result folder:
Path: ../dist/
index.html page generated:
Path: build/deploy/index.html
---------- BUILD FOR TASK ItalianLaws/Test-1 COMPLETE

## Section 9 - Packaging

In [25]:
print(f"---------- FOLDERS SETUP STARTED ----------")

folder_tasks_batch_path = f"{folder_tasks_path}{task_name}/{batch_name}/"
folder_tasks_batch_deploy_path = f"{folder_tasks_batch_path}deploy/"
folder_tasks_batch_mturk_path = f"{folder_tasks_batch_path}mturk/"
folder_tasks_batch_task_path = f"{folder_tasks_batch_path}task/"
folder_tasks_batch_config_path = f"{folder_tasks_batch_path}config/"

if not os.path.exists(folder_tasks_batch_deploy_path):
    os.makedirs(folder_tasks_batch_deploy_path, exist_ok=True)
    print(folder_tasks_batch_deploy_path)
else:
    print("Deploy folder already present")
if not os.path.exists(folder_tasks_batch_mturk_path):
    os.makedirs(folder_tasks_batch_mturk_path, exist_ok=True)
    print(folder_tasks_batch_mturk_path)
else:
    print("MTurk folder already present")
if not os.path.exists(folder_tasks_batch_task_path) and deploy_config:
    os.makedirs(folder_tasks_batch_task_path, exist_ok=True)
    print(folder_tasks_batch_task_path)
else:
    print("Task configuration folder already present")

print(f"---------- FOLDERS SETUP COMPLETED ----------")

Initial Chunk Files | Names         |      Size
main.js             | main          |   4.95 MB
styles.css          | styles        | 144.13 kB
polyfills.js        | polyfills     |  44.17 kB
runtime.js          | runtime       |   1.10 kB
Initial Total |   5.14 MB
Build at: 2021-05-28T14:20:57.314Z - Hash: 435ca07744231163efd1 - Time: 40870ms
Merging Javascript assets to deploy folder:
Processing file: ../dist/Crowd_Frame/main.js
Processing file: ../dist/Crowd_Frame/polyfills.js
Processing file: ../dist/Crowd_Frame/runtime.js
Path: build/deploy/scripts.js
Merging CSS assets to deploy folder:
Processing file: ../dist/Crowd_Frame/styles.css
Path: build/deploy/styles.css
Deleting build result folder:
Path: ../dist/
---------- BUILD FOR TASK ItalianLaws/Test-1 COMPLETED ----------
---------- FOLDERS SETUP STARTED ----------
Deploy folder already present
MTurk folder already present
Task configuration folder already present
---------- FOLDERS SETUP COMPLETED ----------
---------- BUILD FOR